In [1]:
#region Init
print("Begin init")
%run -i init_notebook.ipynb
import tensorflow as tf
%load_ext tensorboard
import pandas as pd

import datetime, os
import random


import numpy as np
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

logs_base_dir = "../.logs"

print("Complete init")
#endregion

Begin init


Complete init


In [2]:
print(tf.config.list_physical_devices('GPU'))

# physical_devices = tf.config.list_physical_devices('GPU')
#
# tf.config.experimental.set_memory_growth(physical_devices[0], True)
# assert tf.config.experimental.get_memory_growth(physical_devices[0])

# Set logs for tf
#tf.debugging.set_log_device_placement(True)

# # # create virtual gpu
# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#    # Create N virtual GPUs with 1GB memory each
#    try:
#      tf.config.experimental.set_virtual_device_configuration(
#          gpus[0],
#          [
#              tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024),
#              tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)
#          ])
#      logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#      print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")
#    except RuntimeError as e:
#      # Virtual devices must be set before GPUs have been initialized
#      print(e)

strategy = tf.distribute.OneDeviceStrategy("/gpu:0")
print("Strategy", strategy)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Strategy <tensorflow.python.distribute.one_device_strategy.OneDeviceStrategy object at 0x000001F8771100A0>


In [3]:
def generate_one_data():
    def calculate_Y(x):
        if (0.2 <= x[0] <= 0.6):
            return 0
        else:
            return 1

    def invert_Y(y):
        if y==0:
            return 1
        else:
            return 0

    x = np.random.rand(5,)
    data = {
        'property1': x,
        'property2': x,
        'y': calculate_Y(x)
    }
    return data

def value_to_tensor(value, x_keys, y_key):
        x = [value.get(key) for key in x_keys]
        y = [1] if value[y_key] == 1 else [0]
        return x, y

def generate_data(n):
    data_example = generate_one_data()
    columns = set(data_example.keys())
    raw = {"date", "close", "open", "date", "high", "loss", "low", "volume", "profit", "loss", "y"}
    x_keys = list(columns - raw)
    x_keys.sort()

    all_x = []
    all_y = []
    for i in range(n):
        value = generate_one_data()
        x, y = value_to_tensor(value, x_keys, "y")
        all_x.append(x)
        all_y.append(y)
    return all_x, all_y

train_x, train_y = generate_data(100000)
test_x, test_y = generate_data(1000)

train_dataset = tf.data.Dataset.from_tensors((train_x, train_y))
test_dataset = tf.data.Dataset.from_tensors((test_x, test_y))

print("DATA is ready")

DATA is ready


In [4]:
from tensorflow import keras

def get_metrics():
    metrics = [
        keras.metrics.FalseNegatives(name="fn"),
        keras.metrics.FalsePositives(name="fp"),
        keras.metrics.TrueNegatives(name="tn"),
        keras.metrics.TruePositives(name="tp"),
        keras.metrics.Precision(name="precision"),
        keras.metrics.Recall(name="recall"),
        keras.metrics.BinaryAccuracy(name="ba")
    ]
    return metrics

def build_model(input_shape):
  model = keras.Sequential([
    keras.layers.Flatten(input_shape=(input_shape)),
    keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(48, activation=tf.nn.relu),
    keras.layers.Dense(48, activation=tf.nn.relu),
    keras.layers.Dense(48, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid),
  ])
  return model

log_dir = os.path.join(logs_base_dir, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir, histogram_freq=1)
print("Save logs to", log_dir)

with strategy.scope():
    model = build_model(input_shape=(2, 5))
    metrics = get_metrics()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
                  loss='binary_crossentropy',
                  metrics=metrics)

    training_history = model.fit(
        train_dataset,
        validation_data = test_dataset,
        epochs=50,
        batch_size=300,
        shuffle=True,
        callbacks=[tensorboard_callback])

print("*"*200)
print("Finished training the model")
validation_result  = model.evaluate(test_dataset, batch_size=300)
print("Finished evaluating the model", validation_result)

Save logs to ../.logs\20210526-233305
Epoch 1/50
1/1 [==============================] - 4s 4s/step - loss: 0.6995 - fn: 42784.0000 - fp: 15678.0000 - tn: 24477.0000 - tp: 17061.0000 - precision: 0.5211 - recall: 0.2851 - ba: 0.4154 - val_loss: 0.6704 - val_fn: 0.0000e+00 - val_fp: 408.0000 - val_tn: 0.0000e+00 - val_tp: 592.0000 - val_precision: 0.5920 - val_recall: 1.0000 - val_ba: 0.5920
Epoch 2/50
1/1 [==============================] - 1s 896ms/step - loss: 0.6690 - fn: 0.0000e+00 - fp: 40155.0000 - tn: 0.0000e+00 - tp: 59845.0000 - precision: 0.5985 - recall: 1.0000 - ba: 0.5985 - val_loss: 0.6693 - val_fn: 0.0000e+00 - val_fp: 408.0000 - val_tn: 0.0000e+00 - val_tp: 592.0000 - val_precision: 0.5920 - val_recall: 1.0000 - val_ba: 0.5920
Epoch 3/50
1/1 [==============================] - 0s 167ms/step - loss: 0.6664 - fn: 0.0000e+00 - fp: 40155.0000 - tn: 0.0000e+00 - tp: 59845.0000 - precision: 0.5985 - recall: 1.0000 - ba: 0.5985 - val_loss: 0.6588 - val_